In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git' &> /dev/null

In [ ]:
from datetime import datetime
import os
import itertools

import json
import itertools
import pandas as pd
import numpy as np
import pycocotools.mask as mask_util
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer

import torch

setup_logger()

In [ ]:
class CFG:
    data_path = '../input/sartorius-cell-instance-segmentation/'
    nfolds = 5
    wfold = 4
    data_folder = '../input/sartorius-cell-instance-segmentation/'
    anno_folder = '/kaggle/working/'
    model_arch = 'mask_rcnn_R_50_FPN_3x.yaml'
    nof_iters = 10000
    seed = 45

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(CFG.seed)

### RLE -> COCO

In [ ]:
def rle_decode(mask_rle, shape):
    """Decode a Run-Length Encoding (RLE) encoded binary mask.

    This function takes an RLE encoded mask as a string and decodes it into a binary mask
    represented as a NumPy array.

    Args:
        mask_rle (str): The RLE encoded mask as a string.
        shape (tuple): The shape of the target binary mask as a tuple (height, width).

    Returns:
        np.ndarray: The decoded binary mask as a NumPy array of shape (height, width).

    Example:
        >>> rle_encoded_mask = "2 5 10 3"
        >>> mask_shape = (5, 7)
        >>> decoded_mask = rle_decode(rle_encoded_mask, mask_shape)

    Note:
        - The input RLE string is expected to consist of pairs of values where the first
          value represents the starting position of a run of ones, and the second value
          represents the length of that run.
        - The output binary mask has ones in the positions specified by the RLE encoding
          and zeros elsewhere.
    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    img = img.reshape(shape)
    return img

In [ ]:
def binary_mask_to_rle(binary_mask):
    """Convert a binary mask to Run-Length Encoding (RLE) format.

    This function takes a binary mask represented as a NumPy array and converts it into
    RLE format, which is a compressed representation of the mask.

    Args:
        binary_mask (np.ndarray): The binary mask as a NumPy array.

    Returns:
        dict: A dictionary containing the RLE-encoded mask with "counts" and "size" fields.

    Example:
        >>> import numpy as np
        >>> binary_mask = np.array([[0, 1, 1], [1, 1, 0]])
        >>> rle_mask = binary_mask_to_rle(binary_mask)

    Note:
        - The input binary mask should be a NumPy array with binary values (0 and 1).
        - The output RLE format consists of two parts: "counts" (run-lengths) and "size" (shape).

    RLE Format:
        The "counts" field is a list of integers representing the run-lengths of ones in the mask.
        The "size" field is a list specifying the shape of the original mask as [height, width].
    """
    rle = {"counts": [], "size": list(binary_mask.shape)}
    counts = rle.get("counts")
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order="F"))):
        if i == 0 and value == 0:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

In [ ]:
def coco_structure(train_df):
    cat_ids = {name: id+1 for id, name in enumerate(train_df.cell_type.unique())}
    cats = [{"name": name, "id": id} for name, id in cat_ids.items()]
    images = [
        {
        "id": id, 
        "width": row.width, 
        "height": row.height, 
        "file_name": f"train/{id}.png"
        } for id, row in train_df.groupby("id").agg("first").iterrows()]
    annotations = []
    for idx, row in tqdm(train_df.iterrows()):
        mk = rle_decode(row.annotation, (row.height, row.width))
        ys, xs = np.where(mk)
        x1, x2 = min(xs), max(xs)
        y1, y2 = min(ys), max(ys)
        enc = binary_mask_to_rle(mk)
        seg = {
            "segmentation": enc,
            "bbox": [int(x1), int(y1), int(x2-x1+1), int(y2-y1+1)],
            "area": int(np.sum(mk)),
            "image_id": row.id,
            "category_id": cat_ids[row.cell_type],
            "iscrowd": 0,
            "id": idx,
        }
        annotations.append(seg)
    return {"categories": cats, "images": images, "annotations": annotations}

In [ ]:
train_df = pd.read_csv(CFG.data_path + 'train.csv')
gkf = GroupKFold(n_splits = CFG.nfolds)

train_df["fold"] = -1
y = train_df.width.values
for f, (t_, v_) in enumerate(gkf.split(X=train_df, y=y, groups=train_df.id.values)):
    train_df.loc[v_, "fold"] = f

fold_id = train_df.fold.copy()

In [ ]:
all_ids = train_df.id.unique()

train_sample = train_df.loc[fold_id != CFG.wfold]
root = coco_structure(train_sample)
with open("annotations_train_f" + str(CFG.wfold) + ".json", 'w', encoding="utf-8") as f:
    json.dump(root, f, ensure_ascii=True, indent=4)

In [ ]:
valid_sample = train_df.loc[fold_id == CFG.wfold]
root = coco_structure(valid_sample)
with open("annotations_valid_f" + str(CFG.wfold) + ".json", 'w', encoding="utf-8") as f:
    json.dump(root, f, ensure_ascii=True, indent=4)

In [ ]:
print("fold" + str(CFG.wfold) + ": produced")

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(CFG.seed)

### Evaluation

In [ ]:
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

In [ ]:
def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

In [ ]:
class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

### Model

In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

    def build_hooks(self):

        # copy of cfg
        cfg = self.cfg.clone()

        # build the original model hooks
        hooks = super().build_hooks()

        # add the best checkpointer hook
        hooks.insert(-1, BestCheckpointer(cfg.TEST.EVAL_PERIOD, 
                                         DetectionCheckpointer(self.model, cfg.OUTPUT_DIR),
                                         "MaP IoU",
                                         "max",
                                         ))
        return hooks

In [ ]:
MetadataCatalog.clear()
DatasetCatalog.clear()

dataDir=Path(CFG.data_folder)
register_coco_instances('sartorius_train',{}, CFG.anno_folder + 'annotations_train_f'+str(CFG.wfold)+'.json', dataDir)
register_coco_instances('sartorius_val',{}, CFG.anno_folder + 'annotations_valid_f'+str(CFG.wfold)+'.json', dataDir)
metadata = MetadataCatalog.get('sartorius_train')
train_ds = DatasetCatalog.get('sartorius_train')

In [ ]:
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/' + CFG.model_arch))
cfg.DATASETS.TRAIN = ("sartorius_train",)
cfg.DATASETS.TEST = ("sartorius_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/' + CFG.model_arch) 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = CFG.nof_iters    
cfg.SOLVER.STEPS = []        
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512    
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .4
cfg.TEST.EVAL_PERIOD = len(DatasetCatalog.get('sartorius_train')) // cfg.SOLVER.IMS_PER_BATCH 

### Train

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()